## INIT

In [1]:
## For google colab only
from google.colab import drive
drive.mount('/content/drive')
import sys, os
## on Phuc's drive
%cd /content/drive/MyDrive/Uni/Thesis/Repo 
os.chdir("/content/drive/MyDrive/Uni/Thesis/Repo")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Uni/Thesis/Repo


In [2]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 14.9 MB/s 
time: 421 µs (started: 2022-11-21 04:36:04 +00:00)


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import shutil
import torch.nn as nn
from skimage import io
import torchvision
import cv2
import sys, os, json, time
import pandas as pd
from json import JSONEncoder
from pandas.io.json import json_normalize
from tqdm.notebook import tqdm
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from albumentations.pytorch.transforms import ToTensorV2
from torchvision import utils
from albumentations import (HorizontalFlip, ShiftScaleRotate, VerticalFlip, Normalize,Flip,
                            Compose, GaussNoise)


time: 4.46 s (started: 2022-11-21 04:36:04 +00:00)


In [4]:
""" PY FILES """
py_file_location = "/content/drive/MyDrive/Uni/Thesis/Repo/utils"
sys.path.append(os.path.abspath(py_file_location))

time: 1.06 ms (started: 2022-11-21 04:36:09 +00:00)


In [5]:
# To avoid non-essential warnings 
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

time: 71.1 ms (started: 2022-11-21 04:36:09 +00:00)


In [6]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version
# Install PyTorch
!pip install torch==1.12.0 torchvision --extra-index-url https://download.pytorch.org/whl/cu113
# Install MMCV
!pip install openmim
!mim install mmcv-full==1.6.0
# !rm -rf mmsegmentation
# !git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |██████████████▌                 | 834.1 MB 1.3 MB/s eta 0:13:04tcmalloc: large alloc 1147494400 bytes == 0x66676000 @  0x7f3277abd615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 237 kB 33.7 MB/s 
     |████████████████████████████████| 51 kB 8.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.1 MB 9.9 MB/s 
     |████████████████████████████████| 190 kB 15.0 MB/s 
/content/drive/MyDrive/Uni/Thesis/Repo/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Uni/Thesis/Repo/mmsegmentation
     |████████████████████████████████| 647 kB 15.5 MB/s 
  Running setup.py develop for mmsegmentation
time: 4min 18s (started: 2022-11-21 04:36:09 +00:00)


In [7]:
from parse_dataset import *
from eval import *
from image_extract import *

load checkpoint from local path: /content/drive/MyDrive/Uni/Thesis/Repo/checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
time: 1min 14s (started: 2022-11-21 04:40:27 +00:00)


## Json file reading

In [8]:
!pwd

/content/drive/MyDrive/Uni/Thesis/Repo/mmsegmentation
time: 174 ms (started: 2022-11-21 04:41:41 +00:00)


In [9]:
# Timestamp tag for check point and best model
from datetime import datetime, timezone
import pytz

def getCTimeStr():
  py_timezone = pytz.timezone('Australia/Adelaide')
  dt_string = datetime.now(py_timezone).strftime("(%H:%M:%S-%b_%d_%Y)")
  # print("date and time =", dt_string)	
  return str(dt_string)

time: 977 µs (started: 2022-11-21 04:41:42 +00:00)


In [10]:
getCTimeStr()

'(15:11:42-Nov_21_2022)'

time: 26.1 ms (started: 2022-11-21 04:41:42 +00:00)


In [11]:
# log dir - create/ check file and paste link below
LOG_DIR = '/content/drive/MyDrive/Uni/Thesis/Repo/log/tree_segmentation.log'

time: 537 µs (started: 2022-11-21 04:41:42 +00:00)


In [12]:
# Generate logger
import logging, time, os

# Create and configure logger
logging.basicConfig(filename=LOG_DIR,filemode='w') 

# Creating an object
logger = logging.getLogger()
 
# Setting the threshold of logger to DEBUG
logger.setLevel(logging.INFO)

# đổi repo link chỗ này
fh = logging.FileHandler(LOG_DIR)
# đổi thresh level - DEBUG < INFO < WARNING < ERROR < CRITICAL 
fh.setLevel(logging.INFO)
logger.addHandler(fh)

time: 3.29 ms (started: 2022-11-21 04:41:42 +00:00)


In [13]:
# logger.info(f'|INFO| {getCTimeStr()}: init')

time: 273 µs (started: 2022-11-21 04:41:42 +00:00)


## Get 1000 tree list

In [14]:
from typing_extensions import dataclass_transform
dtecDS_info = "/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees/detection_datasets.json"
with open(dtecDS_info) as json_data:
  data = json.load(json_data)

time: 1.62 s (started: 2022-11-21 04:42:15 +00:00)


In [15]:
def getDtecDFInfo(ds):
    dsX = pd.DataFrame(list(ds['extended']))
    df = pd.concat([ds,dsX], axis=1)
    # df = df.drop(['Longitude','Latitude','extended'], axis=1)

    return df
dtecDF_Itrain, dtecDF_Itest, dtecDF_Ival = [getDtecDFInfo(pd.DataFrame(data[x])) for x in ('train', 'test', 'val')]
treeSamples = dtecDF_Itrain[['roi','OBJECTID','lat', 'lng']][:18000:18]
treeSamples['panoId'] = ''

time: 150 ms (started: 2022-11-21 04:42:42 +00:00)


In [16]:
treeSamples

,roi,OBJECTID,lat,lng,panoId
0,ROI10,1019,34.146522,-118.147762,
18,ROI7,1061,34.166295,-118.113202,
36,ROI7,1176,34.167332,-118.113183,
54,ROI10,1226,34.142809,-118.147589,
72,ROI5,1277,34.161289,-118.109506,
...,...,...,...,...,...
17910,ROI5,68816,34.157588,-118.117825,
17928,ROI5,68835,34.154293,-118.111659,
17946,ROI12,68938,34.146024,-118.129948,
17964,ROI7,68990,34.161772,-118.112478,


time: 19.4 ms (started: 2022-11-21 04:42:45 +00:00)


In [17]:
data_np =  treeSamples.to_numpy()

time: 3.54 ms (started: 2022-11-21 04:43:02 +00:00)


In [18]:
dsDir = '/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees'
pDs = ParseDataset(dsDir)
panoL = []
for row in data_np:
  lat, lng = row[2:4]
  pID = pDs.streetview_extractor.get_nearest_pano(lat,lng)['Location']['panoId']
  linksPano = pDs.streetview_extractor.get_nearest_pano(lat,lng)['Links']
  logger.info(f'|INFO| {getCTimeStr()}: Set Pano_ {pID} for ({lat},{lng})')

INFO:root:|INFO| (15:13:12-Nov_21_2022): Set Pano_ aAbBg_vCzltynugAdbU9ng for (34.146522093647505,-118.14776230059067)
INFO:root:|INFO| (15:13:13-Nov_21_2022): Set Pano_ r5qEjUL64l1pJMgsgMSC1Q for (34.16629541725228,-118.11320244278134)
INFO:root:|INFO| (15:13:13-Nov_21_2022): Set Pano_ GSAXSZ-j0YE7j9sqdSVbIg for (34.16733170714893,-118.11318345322408)
INFO:root:|INFO| (15:13:14-Nov_21_2022): Set Pano_ jQkcbtJa5divS1AQG1Albg for (34.142809228703385,-118.14758879759127)
INFO:root:|INFO| (15:13:14-Nov_21_2022): Set Pano_ GUa_WJvbtjsfVaS1hGSFiw for (34.16128901440291,-118.1095055690036)
INFO:root:|INFO| (15:13:14-Nov_21_2022): Set Pano_ -ivThutJ13dKaoORtiLIkA for (34.16662761602111,-118.11985281690671)
INFO:root:|INFO| (15:13:14-Nov_21_2022): Set Pano_ ZQP0HjamOrEr4HXHkBQDJg for (34.13774294096177,-118.14723428806359)
INFO:root:|INFO| (15:13:15-Nov_21_2022): Set Pano_ fr--qaqGnFXK0rEGPGq2FQ for (34.13710074412028,-118.14723374446856)
INFO:root:|INFO| (15:13:15-Nov_21_2022): Set Pano_ YKz-

time: 3min 42s (started: 2022-11-21 04:43:09 +00:00)


In [ ]:
!pwd

/content/drive/MyDrive/Uni/Thesis/Repo
time: 153 ms (started: 2022-11-10 05:36:41 +00:00)


In [ ]:
treeSamples['panoId'] = panoL
treeSamples

In [1]:
treeSamples.to_json('tree_M_Samples.json',orient="records")
treeSamples

NameError: ignored

## Tree detection

In [ ]:
!pwd

/content/drive/MyDrive/Uni/Thesis/Repo/mmsegmentation
time: 162 ms (started: 2022-11-10 06:09:16 +00:00)


In [ ]:
pano_info = "/content/drive/MyDrive/Uni/Thesis/Repo/panos.json"

time: 523 µs (started: 2022-11-10 06:38:24 +00:00)


In [ ]:
## Getting pano id list from treeSamples.json
sampleData = pd.read_json('../treeSamples.json')
keyList = list(sampleData['panoId'])

time: 1.02 s (started: 2022-11-10 06:37:54 +00:00)


In [ ]:
""" PANO """
panoDict = dict()
with open(pano_info) as fp:
  panoDict = json.load(fp)

time: 39 s (started: 2022-11-10 06:38:34 +00:00)


In [ ]:
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

time: 801 µs (started: 2022-11-10 06:38:00 +00:00)


In [ ]:
## Testing value
print(f'Index no: {keyList.index("rY6gw0NPF3Y3QmL3dvdjHQ")}')
print(f'ID: {keyList[20000]}')
panoDict['rY6gw0NPF3Y3QmL3dvdjHQ']

'rY6gw0NPF3Y3QmL3dvdjHQ'

In [ ]:
from tempfile import tempdir
import logging

totalP = len(keyList)

# For loop through the dictionary with key
for idx , key in enumerate(keyList):
  pano = panoDict[key]
  # og_segmap, tree_map, bboxes = None, None, None
  # Get data
  
  if 'Detection' in pano:
    # logger.info(f'|INFO| {getCTimeStr()}: {key} detected - {((idx+1)/totalP)*100:.1f}%')
    logger.debug(f'{idx} - {key}')
    continue

  try:
    og_segmap, tree_map, bboxes = get_segmentation_and_bbox(key, 25)
    
  except TimeoutError as te:
    time.sleep(1000)
    og_segmap, tree_map, bboxes = get_segmentation_and_bbox(key, 25)
    logger.error(te)

  except OSError as ose:
    time.sleep(1000)
    try:
      og_segmap, tree_map, bboxes = get_segmentation_and_bbox(key, 25)
      logger.error(ose)
    except FileNotFoundError as fnfe:
      logger.error(fnfe)
      continue

  except FileNotFoundError as fnfe:
    logger.error(fnfe)
    continue

  # Update data to dictionray
  pano.update({'Detection':{
      'og_segmap':og_segmap,
      'tree_concours': tree_map,
      'bboxes': bboxes
  }})

  panoDict.update({key:pano})
  with open(pano_info,'r+') as file:
    json.dump(panoDict,file, cls=NumpyArrayEncoder)
  
  logger.info(f'|INFO| {getCTimeStr()}: {key} detected - {((idx+1)/totalP)*100:.1f}%')

INFO:root:|INFO| (17:21:27-Nov_10_2022): GSAXSZ-j0YE7j9sqdSVbIg detected - 0.3%
INFO:root:|INFO| (17:26:24-Nov_10_2022): jQkcbtJa5divS1AQG1Albg detected - 0.4%
INFO:root:|INFO| (17:31:22-Nov_10_2022): GUa_WJvbtjsfVaS1hGSFiw detected - 0.5%
INFO:root:|INFO| (17:36:24-Nov_10_2022): -ivThutJ13dKaoORtiLIkA detected - 0.6%
INFO:root:|INFO| (17:41:24-Nov_10_2022): ZQP0HjamOrEr4HXHkBQDJg detected - 0.7%
INFO:root:|INFO| (17:46:25-Nov_10_2022): fr--qaqGnFXK0rEGPGq2FQ detected - 0.8%
INFO:root:|INFO| (17:51:28-Nov_10_2022): YKz-J55dtIBdj8NteTXCRw detected - 0.9%
INFO:root:|INFO| (17:56:35-Nov_10_2022): u4_2PuVVAHoAdirhSKETcw detected - 1.0%
INFO:root:|INFO| (18:01:41-Nov_10_2022): nVIDGqg3h3MEPv6K3qYtkg detected - 1.1%
INFO:root:|INFO| (18:06:50-Nov_10_2022): a8vaTFCjRnImcPqJvIzF1Q detected - 1.2%
INFO:root:|INFO| (18:12:03-Nov_10_2022): 7TR-YncEF9INfpduQdF3Bg detected - 1.3%
INFO:root:|INFO| (18:17:13-Nov_10_2022): TSM0sjgtjtFd0JtsbuiGCQ detected - 1.4%
INFO:root:|INFO| (18:22:22-Nov_10_2022):

FileNotFoundError: ignored

time: 7h 46min 10s (started: 2022-11-10 06:46:28 +00:00)


In [ ]:
print(open('../log/detection_report.log').read())

FileNotFoundError: ignored

In [ ]:
# og_segmap, tree_map, bboxes = get_segmentation_and_bbox(list(panoDict.keys())[1], 20)